In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
directory = r"C:\Users\Meyer\Documents\Dom_Detecter\UNSW-csv_03-04"  # Replace with your directory path
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [4]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

C:\Users\Meyer\AppData\Local\Temp\ipykernel_14448\2217386177.py:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]


In [5]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [5]:
df_merged = merged_dataframe

In [6]:
df_merged.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,Start_Time,End_Time,Duration,Flow_Volume,Flow_Rate,Inter_Arrival_Time,Total_Fwd_Packets,Total_Bwd_Packets,Fwd_Packet_Length_Total,Bwd_Packet_Length_Total
3015146,2344987,UDP,192.168.1.240,47532.0,23.23.189.20,33434.0,191.0,NaN,NaN,64.0,...,1.475500e+09,1.475586e+09,86377.446387,603981,6.992346,27.028617,3159,2,603369,612
3015147,2344988,TCP,192.168.1.106,47940.0,52.87.241.159,443.0,156.0,ack,2549.0,64.0,...,1.475500e+09,1.475586e+09,86396.665415,29968618,346.872392,0.808389,114406,113720,22253830,7714788
3015148,2344989,TCP,89.30.121.22,5222.0,192.168.1.156,40179.0,195.0,ack,185856.0,44.0,...,1.475523e+09,1.475586e+09,63310.846057,1546822,24.432180,0.272993,6261,3482,911350,635472
3015149,2344990,TCP,192.168.1.156,40179.0,89.30.121.22,5222.0,66.0,ack,126160.0,64.0,...,1.475523e+09,1.475586e+09,63310.846057,1546822,24.432180,0.002094,6261,3482,911350,635472
3015150,2344991,NaN,fe80::216:6cff:feab:6b88,NaN,ff02::1:ffab:6b88,NaN,NaN,NaN,NaN,1.0,...,0.000000e+00,0.000000e+00,0.000000,0,0.000000,1.279176,0,0,0,0


In [7]:
# Count the number of rows
num_rows = df_merged.shape[0]

# Print the number of rows
print(f"The DataFrame has {num_rows} rows.")

The DataFrame has 3015151 rows.


In [8]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\UNSW_csv_23-02\mapping\mapping.xlsx', engine='openpyxl')

In [9]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.packet_eth_src, mapping_df.target))

In [10]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['packet_eth_src'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [11]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               2617678
samsung_cam             75405
belkin_motion           70803
belkin_switch           59506
amazon_echo             57436
withings_sleep          30498
netatmo_cam             29845
smart_things            23918
tp-link_cam             11350
lifx                     9117
netatmo_weather          7161
triby_speaker            6973
ihome_plug               5867
hp_printer               4200
pixstar_photoframe       2674
tp-link_plug             2245
withings_scale            231
nest_smoke                218
blipcare_bp_meter          26
Name: count, dtype: int64


In [13]:
flow_counts = df_merged.groupby('target')['Flow'].nunique()
print(flow_counts)

target
amazon_echo            4449
belkin_motion          4014
belkin_switch          2926
blipcare_bp_meter         3
hp_printer               12
ihome_plug               19
lifx                   1447
nest_smoke               15
netatmo_cam             947
netatmo_weather         248
pixstar_photoframe      405
samsung_cam            3555
smart_things            584
tp-link_cam             405
tp-link_plug            176
triby_speaker            27
unknown               24569
withings_scale            4
withings_sleep         1183
Name: Flow, dtype: int64


In [15]:
# List of targets you want to keep
desired_targets = ['amazon_echo', 'belkin_motion', 'belkin_switch', 'netatmo_cam', 
                   'netatmo_weather', 'pixstar_photoframe', 'samsung_cam', 
                   'smart_things', 'tp-link_cam', 'triby_speaker', 'unknown']

# Filtering the DataFrame
filtered_df = df_merged[df_merged['target'].isin(desired_targets)]

In [16]:
# Count the number of packets for each target and print it
target_counts = filtered_df['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               2617678
samsung_cam             75405
belkin_motion           70803
belkin_switch           59506
amazon_echo             57436
netatmo_cam             29845
smart_things            23918
tp-link_cam             11350
netatmo_weather          7161
triby_speaker            6973
pixstar_photoframe       2674
Name: count, dtype: int64


In [17]:
def limit_rows(group):
    return group.head(20000)

# Assuming df_merged is your DataFrame
limited_df = filtered_df.groupby('target').apply(limit_rows).reset_index(drop=True)


In [19]:
# Count the number of packets for each target and print it
target_counts = limited_df['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
amazon_echo           20000
belkin_motion         20000
belkin_switch         20000
netatmo_cam           20000
samsung_cam           20000
smart_things          20000
unknown               20000
tp-link_cam           11350
netatmo_weather        7161
triby_speaker          6973
pixstar_photoframe     2674
Name: count, dtype: int64


In [26]:
limited_df.to_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\limited_and_filtered_df_unsw_02_04.xlsx', index=False)

In [ ]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)